In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [ ]:
df = pd.read_csv('Feature_Extracition.csv', index_col= 'Unnamed: 0')
df.head()

# mapping

In [ ]:
df['label'] = df['SOFA score'].apply(lambda x: 0 if x < 12 else 1)
df.head()

In [ ]:
sns.countplot(data= df, x= 'label')

In [ ]:
plt.figure(figsize= (39, 15), facecolor='white')
for i, col in enumerate(df):
    plt.subplot(5, 13, i+1)
    sns.boxplot(data= df, x= 'label', y = f'{col}')
    plt.title(col)

plt.tight_layout()
plt.show()

# feature selection + Data split

In [ ]:
from sklearn.model_selection import StratifiedKFold

# StarifiedKfold
def Starifieds(df_feature_cl, pure_feature, random_state= 42, n_splits= 4, shuffle= True):

    #======================================
    # 用處: Starifieds Kflod 原本只能出index，這裡二合一
    #
    # Parameter: 
    #     df_feature_cl, feature
    #     pure_feature, label
    #     random_state= 42, 不必多說
    #     n_splits= 4,  分幾份
    #     shuffle= True  如字面上
    #======================================

    # 先創StratifiedKFold 然後再 .split() 然後取第一個
    xtrain, xtest = list(StratifiedKFold(n_splits= n_splits, shuffle= shuffle, random_state= random_state).split(df_feature_cl, pure_feature))[0]

    # 把index(用iloc給row的方式)
    train_fea, test_fea = df_feature_cl.iloc[xtrain], df_feature_cl.iloc[xtest]
    train_label, test_label = pure_feature.iloc[xtrain], pure_feature.iloc[xtest]

    return train_fea, train_label, test_fea, test_label

In [ ]:
X = df.drop(columns= ['label', 'SOFA score'])
label = df['label']


x_train, label_train, x_test, label_test = Starifieds(X, label, shuffle=True, random_state=42)

label_train, label_test

# ttest

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import ParameterSampler
from sklearn.metrics import accuracy_score, recall_score
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier

In [ ]:
import scipy.stats
# feature selection

def ttttest(train_fea, train_label):

    #======================================================
    # 1. scipy.stats.shapiro
    shap_low = []
    shap_high = []
    shap_cols = []

    all_fea = train_fea.merge(train_label, left_index= True, right_index= True)
    all_fea = all_fea.sort_values(by= train_label.name)

    # print(train_label.name)

    cols = all_fea.columns
    #print(cols)

    all_low = all_fea[all_fea[train_label.name] == 0]
    all_high = all_fea[all_fea[train_label.name] == 1]

    for col in cols:
        # 看p_value 可不留
        lows = scipy.stats.shapiro(all_low[col])[1]
        highs = scipy.stats.shapiro(all_high[col])[1]

        shap_low.append(lows)
        shap_high.append(highs)

        if lows > 0.05 and highs > 0.05:
            shap_cols.append(col)

    shap_cols
    #=====================================================
    # 2.scipy.stats.levene

    levene = []
    good_levene = []

    for shapiro_col in shap_cols:

        lev = scipy.stats.levene(all_low[shapiro_col], all_high[shapiro_col], center = 'mean')[1]  
        levene.append(lev)

        if lev > 0.05:
            good_levene.append(shapiro_col)

    good_levene

    #======================================================
    # 3. scipy.stats.ttest_ind
    
    ttest = []
    good_ttest = []


    for good_lev in good_levene:
        ttestn = scipy.stats.ttest_ind(all_low[good_lev], all_high[good_lev], equal_var = True)[1]
        ttest.append(ttestn)

    #display results
    ttest_df = pd.Series(ttest, index= good_levene, name= train_label.name + '_T_score').sort_values()
    return ttest_df



In [ ]:
def acc_plot(test_feature, test_label, train_fea, train_label,  model, h = 0.02, bound= 1,  **params):

    # plot function
    #===========================================#
    #用途: 畫出邊界plot
    
    #切記: 二維才能畫圖
    #===========================================#
    # Import: 
    # numpy
    # matplotlib.pyplot
    #===========================================#

    # 白底
    plt.style.use('seaborn-white')
    sns.set(font_scale=1.4)

    # 確定是二維
    if len(test_feature.columns) == 2:
        # 把網格架好
        def make_meshgrid(x1, x2, h = h):
            x_min, x_max = x1.min() - bound, x1.max() + bound
            y_min, y_max = x2.min() - bound, x2.max() + bound
            xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
            return xx, yy
        
        # 畫出等高線
        def plot_contours(clf, xx, yy, ax, **params ):
            z = clf.predict(np.c_[xx.ravel(), yy.ravel()])  #np.c_ 帥爛
            z = z.reshape(xx.shape)
            ax.contourf(xx, yy, z, alpha= 0.8, cmap = plt.cm.coolwarm)
            ax.set_xlim(xx.min(), xx.max())
            ax.set_ylim(yy.min(), yy.max())
                        

        fig, (ax1, ax2) = plt.subplots(1, 2, figsize= (12, 6))

        # 要用test的 還是train的
        xx, yy = make_meshgrid(test_feature.iloc[:, 0],  test_feature.iloc[:, 1])
        

        plot_contours(model, xx, yy, cmap=plt.cm.coolwarm, ax= ax1, **params)
        ax1.scatter(train_fea.iloc[:, 0], train_fea.iloc[:, 1], c= train_label, cmap=plt.cm.coolwarm)  # 用label拚座標，暈爛
        ax1.set_xlabel(train_fea.columns[0])
        ax1.set_ylabel(train_fea.columns[1])
        ax1.set_title(f"{model.__class__.__name__} Train Plot {accuracy_score(train_label, model.predict(train_fea))}")


        plot_contours(model, xx, yy, cmap=plt.cm.coolwarm, ax= ax2,  **params)
        ax2.scatter(test_feature.iloc[:, 0], test_feature.iloc[:, 1], c= test_label, cmap=plt.cm.coolwarm)  # 用label拚座標，暈爛
        ax2.set_xlabel(test_feature.columns[0])
        ax2.set_ylabel(test_feature.columns[1])
        ax2.set_title(f"{model.__class__.__name__} Test Plot {accuracy_score(test_label, model.predict(test_feature))}")
        
        
        

        plt.tight_layout()
        plt.show()

    else: 
        print("Data should be two dimension!!")

        # pass

    #==================R=O=C===================================================#

In [ ]:
def condusion_m(test_fea, test_label, model):

    #=============================
    # 用途: 
    #     製造出confusion matrix

    # Parameter:
    #     test_fea  就feature
    #     test_label  就label
    #     model  就model
    #=============================

    from sklearn.metrics import confusion_matrix
    
    conf = confusion_matrix(test_label, model.predict(test_fea))
    conp = np.array([(x/sum(x)) for x in conf])

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize= (14, 7))
    # 重點是 annot，cmap='Blues'
    sns.set(font_scale=1.8)
    sns.heatmap(conp , annot= True, cmap='Blues', ax= ax1)
    ax1.set_title(f"CM of {model.__class__.__name__}")

    sns.heatmap(conf , annot= True, cmap='Blues', ax= ax2)
    ax2.set_title(f"CM of {model.__class__.__name__}")
    plt.tight_layout()
    plt.show()

In [ ]:
def ROCP(test_fea, test_label, model, a= 0, pos_label= 0):
    from sklearn.metrics import roc_curve, auc

    #=============================
    # 用途: 
    #     製造出confusion matrix

    # Parameter:
    #     test_fea  就feature
    #     test_label  就label
    #     model  就model
    #     a= 0 如果auc很奇怪，就 =1 
    #=============================

    try:
        prob = model.predict_proba(test_fea)[:, a]
        sns.set(font_scale=1.4)
        fig, ax= plt.subplots(1, 1, figsize= (8, 7))

        
        
        
        fpr, tpr, thresholds = roc_curve(test_label, prob, pos_label= pos_label)  #pos_label= 0 要設值
        plt.plot(fpr, tpr,   color= 'b', linewidth=3.0)

        
        x, y = np.arange(0, 1, 0.01), np.arange(0, 1, 0.01)
        plt.plot(x, y, '-.', linewidth=3.0, label= f"AUC  = {auc(fpr, tpr)}", color= 'r')
        
        ax.set_xlabel(" 1 - specificity")
        ax.set_ylabel("Sensitivity")
        ax.set_title(f"{model.__class__.__name__}'s  ROC")
        plt.legend()
        plt.show()
    
    except:
        print("Can't print ROC")

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import ParameterSampler
from sklearn.metrics import accuracy_score
from sklearn.linear_model import SGDClassifier
from itertools import combinations

for iui in range(1000):
    print(iui, end= " ")
    train_feas1, train_labels, test_feas1, test_labels = Starifieds(X, label, random_state= iui)    
# try:
    cbi2 = ttttest(train_feas1, train_labels)
    goods = cbi2[cbi2 < 0.05]
    
    # print(goods)
    comb = combinations(goods.index, 2)
    # print(list(comb))
    

    # if cbi2[0] < 0.05 and cbi2[1] < 0.05:
    for good_feature in list(comb):     
           
        train_feas, test_feas = train_feas1[list(good_feature)], test_feas1[list(good_feature)]    

        # ====================
        # print(train_feas.columns)
        for imm in train_feas.columns:
            if 'std' not in imm:     
                sd = StandardScaler()
                train_feasn = sd.fit_transform(train_feas[imm].values.reshape(-1,1))
                test_feasn = sd.transform(test_feas[imm].values.reshape(-1,1))

                train_feas[imm] = train_feasn
                test_feas[imm] = test_feasn

                # print(f'trans {imm}')
        # print(test_feas)
        # ===================================

        
        param_grid = {"C": [0.01, 0.1, 1, 10, 100], "gamma":[0.01, 0.1, 1, 5]}

        param_list = list(ParameterSampler(param_grid, n_iter=10000))

        for par in param_list:
            csc = SVC(probability=True, **par)                          
        
        #gc = GridSearchCV(csc, params, cv= 5, n_jobs= -1, scoring= 'accuracy')
            csc.fit(train_feas, train_labels)
            train_s = accuracy_score(train_labels, csc.predict(train_feas))
            test_s = accuracy_score(test_labels, csc.predict(test_feas))
            
            recalls = recall_score(test_labels, csc.predict(test_feas))

        # pred = gc.best_estimator_.predict(test_feas)
            if test_s  >= 0.79 and train_s  >= 0.75 and test_s < 1 and train_s < 1 and recalls >= 0.68:
                print('rand = ', iui)
                print(cbi2.loc[list(good_feature)])
                
                # print("train ", gc.best_score_)
                print('train ', train_s)
                print('test', test_s)
                print("par = ", par)

                
                print('Recall ', recalls)

                print(goods)

                acc_plot(test_feas, test_labels, train_feas, train_labels, csc)
                condusion_m(test_feas, test_labels, csc)
                ROCP(test_feas, test_labels, csc, a= 0)
    # except:
    #     pass

# 做一次

In [ ]:
iui = 750
good_feature = ['tHb_begin_slope_stage1', 'HHb_mean_diff_stage2']
par =  {'gamma': 1, 'C': 10}

train_feas1, train_labels, test_feas1, test_labels = Starifieds(X, label, random_state= iui)    

cbi2 = ttttest(train_feas1, train_labels)
goods = cbi2[cbi2 < 0.05]
    
train_feas, test_feas = train_feas1[list(good_feature)], test_feas1[list(good_feature)]    

# ====================
# print(train_feas.columns)
for imm in train_feas.columns:
    if 'std' not in imm:     
        sd = StandardScaler()
        train_feasn = sd.fit_transform(train_feas[imm].values.reshape(-1,1))
        test_feasn = sd.transform(test_feas[imm].values.reshape(-1,1))

        train_feas[imm] = train_feasn
        test_feas[imm] = test_feasn

        # print(f'trans {imm}')
# print(test_feas)
# ===================================




csc = SVC(probability=True, **par)                          

#gc = GridSearchCV(csc, params, cv= 5, n_jobs= -1, scoring= 'accuracy')
csc.fit(train_feas, train_labels)
train_s = accuracy_score(train_labels, csc.predict(train_feas))
test_s = accuracy_score(test_labels, csc.predict(test_feas))

recalls = recall_score(test_labels, csc.predict(test_feas))


print('rand = ', iui)
print(cbi2.loc[list(good_feature)])

# print("train ", gc.best_score_)
print('train ', train_s)
print('test', test_s)
print("par = ", par)


print('Recall ', recalls)

print(goods)

acc_plot(test_feas, test_labels, train_feas, train_labels, csc)
condusion_m(test_feas, test_labels, csc)
ROCP(test_feas, test_labels, csc, a= 1)

# 看特徵是否長不一樣

In [ ]:
wanted = df[['tHb_begin_slope_stage1', 'HHb_mean_diff_stage2', 'label']]
wanted.head()

In [ ]:
plt.figure(figsize=(8, 8))
sns.boxplot(data=wanted, x= 'label', y= 'tHb_begin_slope_stage1')
plt.xticks([0, 1], ['Low', 'High'])
plt.xlabel('Group')
plt.show()

In [ ]:
plt.figure(figsize=(8, 8))
sns.boxplot(data=wanted, x= 'label', y= 'HHb_mean_diff_stage2')
plt.xticks([0, 1], ['Low', 'High'])
plt.xlabel('Group')
plt.show()